In [16]:
import numpy as np
import pandas as pd

In [158]:
Holidays = pd.read_excel('Holidays.xlsx')
Shipment = pd.read_csv('Shipment_upto35.csv')

In [36]:
Holidays.head(4)

,Date,Day_week,Holiday,isHoliday,isObserved,Area
0,2010-01-01,Friday,New Year's Day,1.0,NaN,NaN
1,2010-01-02,Saturday,Ordinary,NaN,NaN,NaN
2,2010-01-03,Sunday,Ordinary,NaN,NaN,NaN
3,2010-01-04,Monday,2nd January,1.0,NaN,Scotland


In [14]:
Shipment.head(3)

,APG,APG:FU,Actuals,DP Lag4,FU,Week
0,UGB001,UGB001##3100:IGB0007,1170.0,1572.5,IGB0007,201601
1,UGB001,UGB001##3100:IGB0007,1950.0,2212.9,IGB0007,201602
2,UGB001,UGB001##3100:IGB0007,2145.0,1459.5,IGB0007,201603


In [171]:
Shipment['Week'] = Shipment['Week'].astype(str)
Holidays.Date = Holidays.Date.astype(str)

In [170]:
Holidays.dtypes

Date          datetime64[ns]
Day_week              object
Holiday               object
isHoliday            float64
isObserved           float64
Area                  object
YearWeek              object
Week                  object
dtype: object

In [ ]:
Holidays['Week'] = pd.to_datetime(Holidays['Date']).dt.strftime('%G%V')

In [169]:
from datetime import datetime
Holidays['Week'] =  Holidays['Date'].map(lambda x: x.strftime('%G%V'))

In [67]:
def year_week(column):
    week_number = column.map(lambda x: ((x - datetime.datetime(x.year, 1, 1)).days // 7) + 1).astype(str)
    week_number = week_number.map(lambda x: x if len(x) > 1 else '0' + x)
    return column.map(lambda x: x.year).astype(str) + week_number   

In [93]:
Holidays['Week'] = year_week(Holidays.Date)

In [72]:
Holidays.head(3)

,Date,Day_week,Holiday,isHoliday,isObserved,Area,Week
0,2010-01-01,Friday,New Year's Day,1.0,NaN,NaN,201001
1,2010-01-02,Saturday,Ordinary,NaN,NaN,NaN,201001
2,2010-01-03,Sunday,Ordinary,NaN,NaN,NaN,201001


### Merge dataframes:

In [172]:
left_merge = pd.merge(Shipment, Holidays, on ='Week', how = 'left')
right_merge = pd.merge(Shipment, Holidays, on ='Week', how = 'right')
inner_merge = pd.merge(Shipment, Holidays, on ='Week', how = 'inner')
outer_merge = pd.merge(Shipment, Holidays, on ='Week', how = 'outer')

In [185]:
Holidays.columns

Index(['Date', 'Day_week', 'Holiday', 'isHoliday', 'isObserved', 'Area',
       'YearWeek', 'Week', 'Week2'],
      dtype='object')

In [177]:
Shipment.shape

(351457, 7)

In [186]:
left_merge = pd.merge(Shipment, Holidays, left_on = 'Week', right_on = 'Week2', how = 'left')

In [190]:
len(Shipment.Week)

351457

In [192]:
len(Shipment.Week.unique())

139

In [211]:
week_counts = Holidays.groupby('Week').Week.count().to_frame()

In [231]:
Holidays.groupby('Week').

In [233]:
{c: list(Shipment[c].unique()) for c in Shipment.columns}

{'APG': ['UGB001',
  'UGB002',
  'UGB003',
  'UGB004',
  'UGB005',
  'UGB006',
  'UGB007',
  'UGB008',
  'UGB009',
  'UGB010',
  'UGB011',
  'UGB012',
  'UGB013',
  'UGB014',
  'UGB015',
  'UGB016',
  'UGB017',
  'UGB999'],
 'APG:FU': ['UGB001##3100:IGB0007',
  'UGB001##3100:IGB0008',
  'UGB001##3100:IGB0009',
  'UGB001##3100:IGB0012',
  'UGB001##3100:IGB0013',
  'UGB001##3100:IGB0014',
  'UGB001##3100:IGB0015',
  'UGB001##3100:IGB0026',
  'UGB001##3100:IGB0033',
  'UGB001##3100:IGB0040',
  'UGB001##3100:IGB0041',
  'UGB001##3100:IGB0042',
  'UGB001##3100:IGB0043',
  'UGB001##3100:IGB0060',
  'UGB001##3100:IGB0061',
  'UGB001##3100:IGB0084',
  'UGB001##3100:IGB0099',
  'UGB001##3100:IGB0100',
  'UGB001##3100:IGB0137',
  'UGB001##3100:IGB0138',
  'UGB001##3100:IGB0140',
  'UGB001##3100:IGB0156',
  'UGB001##3100:IGB0157',
  'UGB001##3100:IGB0158',
  'UGB001##3100:IGB0169',
  'UGB001##3100:IGB0170',
  'UGB001##3100:IGB0185',
  'UGB001##3100:IGB0186',
  'UGB001##3100:IGB0193',
  'UGB001##3

In [234]:
Shipment.value_counts()

AttributeError: 'DataFrame' object has no attribute 'value_counts'

In [ ]:
pd.unique(df1[['Col1', 'Col2']].values.ravel('K'))

In [ ]:
df[['Col1', 'Col2']].apply(lambda x: ''.join(x), axis=1)

In [ ]:
week_counts = Holidays.groupby('Week').Week.count().to_frame()

In [216]:
week_counts = week_counts.rename(columns={"Week": "Week_qty"}).reset_index()

In [227]:
Shipment = Shipment.merge(week_counts, on='Week', how = 'left')

In [224]:
len(Shipment)

351457

In [230]:
Shipment.Week_qty.unique()

array([7, 8], dtype=int64)

In [ ]:
left_merge = pd.merge(Shipment, Holidays[['Date', 'Day_week', 'Holiday']], left_on ='Week', right_on = 'Week2', how = 'left')

In [ ]:
pd.merge(Shipment, Holidays

In [188]:
left_merge.shape

(2462639, 16)

In [181]:
2 462 639/2  351 457

1231319.5

### Diff column to merge:

In [175]:
Holidays['Week2'] = Holidays.Week

In [ ]:
left_merge2 = pd.merge(Shipment, Holidays[['Date', 'Day_week', 'Holiday']], left_on ='Week', right_on = 'Week2', how = 'left')
right_merge2 = pd.merge(Shipment, Holidays[['Date', 'Day_week', 'Holiday']], left_on ='Week', right_on = 'Week2', how = 'right')
inner_merge2 = pd.merge(Shipment, Holidays[['Date', 'Day_week', 'Holiday']], left_on ='Week', right_on = 'Week2', how = 'inner')
outer_merge2 = pd.merge(Shipment, Holidays[['Date', 'Day_week', 'Holiday']], left_on ='Week', right_on = 'Week2', how = 'outer')

## Merge with 2 column:

In [174]:
Shipment['Week2'] = Shipment.Week

In [176]:
left_merge = pd.merge(Shipment, Holidays, on = ['Week', 'Week2'], how = 'left')
right_merge = pd.merge(Shipment, Holidays, on = ['Week', 'Week2'], how = 'right')
inner_merge = pd.merge(Shipment, Holidays, on = ['Week', 'Week2'], how = 'inner')
outer_merge = pd.merge(Shipment, Holidays, on = ['Week', 'Week2'], how = 'outer')

## Merge with 2 indexes:

In [108]:
# create indexes for example:
Shipment.set_index(['Week', Shipment.Week2.map(lambda x: x[0:4])], inplace = True)
Holidays.set_index(['Week', Holidays.Week2.map(lambda x: x[0:4])], inplace = True)

In [113]:
Shipment.index.names = ['Week', 'Year']
Holidays.index.names = ['Week', 'Year']

In [123]:
Shipment.head(5)

,,APG,APG:FU,Actuals,DP Lag4,FU,Week2
Week,Year,,,,,,
201601,2016,UGB001,UGB001##3100:IGB0007,1170.0,1572.5,IGB0007,201601
201602,2016,UGB001,UGB001##3100:IGB0007,1950.0,2212.9,IGB0007,201602
201603,2016,UGB001,UGB001##3100:IGB0007,2145.0,1459.5,IGB0007,201603
201604,2016,UGB001,UGB001##3100:IGB0007,1170.0,2162.3,IGB0007,201604
201605,2016,UGB001,UGB001##3100:IGB0007,2340.0,8097.5,IGB0007,201605


In [125]:
index_merge = pd.merge(Shipment, Holidays, left_index=True, right_index=True)

### Join the two dataframes along rows:

In [79]:
df_new = pd.concat([Holidays[Holidays.Day_week == 'Friday'], Holidays[Holidays.Day_week == 'Sunday']])

### 3 dataframes:

In [81]:
# create third table
Holidays2 = Holidays[Holidays.Day_week == 'Friday']

In [82]:
three_df = Shipment.merge(Holidays, on='Week').merge(Holidays2, on='Week')

In [ ]:
group by aggregate aggregation multiple columns 
groupby transform python